<a href="https://colab.research.google.com/github/RebecaGis/Py/blob/main/Geocodifica%C3%A7%C3%A3o_Completa_Rua%2C_N%C3%BAmero%2C_Bairro%2C_Cidade_e_CEP_OSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arcpy
import requests
import time

# Configurações
shapefile = r"C:CAMINHO/DO/ARQUIVO/AQUI"
campo_lat = "Latitude"      # substitua pelo nome real do campo
campo_lon = "Longitude"     # substitua pelo nome real do campo
campo_endereco = "Endereço" # campo a ser preenchido

# Verificar se o campo Endereço existe, se não, criar
if campo_endereco not in [field.name for field in arcpy.ListFields(shapefile)]:
    arcpy.AddField_management(shapefile, campo_endereco, "TEXT", field_length=255)

# Função de geocodificação reversa usando Nominatim
def get_address(lat, lon):
    try:
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1"
        headers = {'User-Agent': 'PDAU-Geocoding/1.0'}
        response = requests.get(url, headers=headers).json()
        address = response.get('address', {})

        rua = address.get('road') or address.get('pedestrian', '')
        numero = address.get('house_number', '')
        bairro = address.get('suburb') or address.get('neighbourhood', '')
        cidade = address.get('city') or address.get('town', '')
        cep = address.get('postcode', '')

        componentes = [rua, numero, bairro, cidade, cep]
        endereco = ', '.join([c for c in componentes if c])
        return endereco if endereco else "Endereço não encontrado"
    except Exception as e:
        print(f"Erro ao geocodificar ({lat}, {lon}): {str(e)}")
        return "Erro na geocodificação"

# Atualizar todas as feições
with arcpy.da.UpdateCursor(shapefile, [campo_lat, campo_lon, campo_endereco]) as cursor:
    contador = 0
    for row in cursor:
        lat, lon = row[0], row[1]
        if not row[2] or row[2] in ("", "Endereço não encontrado", "Erro na geocodificação"):
            if lat and lon:
                print(f"Processando {contador + 1}...")
                endereco = get_address(lat, lon)
                row[2] = endereco
                cursor.updateRow(row)
                contador += 1
                time.sleep(1.2)  # Nominatim requer mínimo de 1 segundo entre requisições
            else:
                row[2] = "Coordenadas ausentes"
                cursor.updateRow(row)

print(f"\nProcesso concluído! {contador} endereços foram geocodificados.")
